In [ ]:
#  Installing all requirements
!pip install requests pandas

#  Importing all libraries
import requests
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report




In [ ]:
# Parsing into DataFrame
job_titles = []
record_ids = []

for record in records:
    if 'Job Title' in record['fields']:  # ensuring direct match in airtable field
        job_titles.append(record['fields']['Job Title'])
        record_ids.append(record['id'])

df = pd.DataFrame({
    "Job Title": job_titles,
    "RecordID": record_ids
})

print("Number of rows fetched:", len(df))
display(df.head(10))


Number of rows fetched: 411


,Job Title,RecordID
0,"Senior Director, Corporate Security @Verizon",rec06mTVFx0w9kNdV
1,Global Capital Solutions @ Ares Management Cor...,rec06pz9a2yyinKyL
2,"Data Engineer @ Casual Precision, LLC",rec08cCaKhKKmSlAx
3,Sr. HR Administrative Coordinator @Loyola Mary...,rec0lwAgo2xvuetjA
4,"Senior Analyst, Sales Relationship Specialist ...",rec0nI89qZwTMByk9
5,Systems Consultant/Techincal Trainer @Cities D...,rec0oBYbJ9ed2YJg2
6,Producer @ C3 Risk & Insurance Services,rec0qW2RclMkhherl
7,Senior Business Systems Analyst @ Native Foods...,rec0uinUEmalfp4Im
8,Sr. Data Analyst @Origence,rec0zeci28P9qv0Mv
9,Software Engineer @Datavant,rec13GdtIpBi4BuJy


In [ ]:
import requests
import pandas as pd

AIRTABLE_TOKEN = "your_airtable_token_here"
BASE_ID = "app59RhWQEcM6gMLt"
TABLE_ID = "tblbgZIIhuhPJD3yC"
url = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_ID}"

headers = {
    "Authorization": f"Bearer {AIRTABLE_TOKEN}",
    "Content-Type": "application/json"
}

# Fetching records
records = []
offset = None

while True:
    params = {"offset": offset} if offset else {}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    print("DEBUG RESPONSE:", data)  # In response to KeyError

    records.extend(data["records"])
    offset = data.get("offset")
    if not offset:
        break


DEBUG RESPONSE: {'records': [{'id': 'rec06mTVFx0w9kNdV', 'createdTime': '2025-03-25T23:25:46.000Z', 'fields': {'Full Name': 'Anshu Arora', 'FieldOfStudy': 'Management Information Systems', 'Graduation Year': 1998, 'Job Title': 'Senior Director, Corporate Security @Verizon', 'LinkedIn Profile': 'https://www.linkedin.com/in/anshu-arora-4393822/', 'Last Parsed Time': '2025-04-10T17:04:46.346Z', 'PredictedCategory': 'Business', 'Created By': {'id': 'usroNBW4DWw4DsS9J', 'email': 'jzimme20@lion.lmu.edu', 'name': 'James Zimmerman'}, 'Last Name': 'Arora', 'First Name': 'Anshu'}}, {'id': 'rec06pz9a2yyinKyL', 'createdTime': '2025-03-25T23:25:46.000Z', 'fields': {'Full Name': 'Max Kaiser', 'FieldOfStudy': 'Business Analytics ', 'Graduation Year': 2021, 'Job Title': 'Global Capital Solutions @ Ares Management Corporation', 'LinkedIn Profile': 'https://www.linkedin.com/in/maxkaiser1/', 'Rating': 'Great', 'Last Parsed Time': '2025-04-10T17:04:46.550Z', 'Created By': {'id': 'usroNBW4DWw4DsS9J', 'emai

In [ ]:
print("DEBUG RESPONSE:\n", data)


DEBUG RESPONSE:
 {'records': [{'id': 'recywnRRiXDfevw0V', 'createdTime': '2025-03-25T23:25:46.000Z', 'fields': {'Full Name': 'Brian Lynch', 'FieldOfStudy': 'Management Information Systems', 'Graduation Year': 1991, 'Job Title': 'CFO @BottleOne', 'LinkedIn Profile': 'https://www.linkedin.com/in/balynch/', 'Last Parsed Time': '2025-04-10T17:06:18.613Z', 'PredictedCategory': 'Media', 'Created By': {'id': 'usroNBW4DWw4DsS9J', 'email': 'jzimme20@lion.lmu.edu', 'name': 'James Zimmerman'}, 'Last Name': 'Lynch', 'First Name': 'Brian'}}, {'id': 'recz0tu4ZuacOm2AP', 'createdTime': '2025-03-25T23:25:46.000Z', 'fields': {'Full Name': 'Tyler Kinomoto', 'FieldOfStudy': 'Information Systems Business Analytics', 'Graduation Year': 2022, 'Job Title': 'Analyst @ Endeavor', 'LinkedIn Profile': 'https://www.linkedin.com/in/tyler-kinomoto/', 'Rating': 'Great', 'Last Parsed Time': '2025-04-10T17:06:18.799Z', 'PredictedCategory': 'Business', 'Created By': {'id': 'usroNBW4DWw4DsS9J', 'email': 'jzimme20@lion.l

In [ ]:
# Fetch records
records = []
offset = None

while True:
    params = {"offset": offset} if offset else {}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    records.extend(data["records"])
    offset = data.get("offset")
    if not offset:
        break

# Parsing into DataFrame
job_titles = []
record_ids = []

for record in records:
    if 'Job Title' in record['fields']:  # ✅ correct field name
        job_titles.append(record['fields']['Job Title'])
        record_ids.append(record['id'])

df = pd.DataFrame({
    "Job Title": job_titles,
    "RecordID": record_ids
})

print("Number of rows fetched:", len(df))
print("First 5 rows:\n", df.head())



# Show the top 10 for now (temp)
display(df.head(10))
# Assigning categories based on keywords in Job Titles
def assign_category(title):
    title = str(title).lower()

    if any(kw in title for kw in ["software", "engineer", "developer", "it", "cyber", "security", "cloud", "technical"]):
        return "Tech"
    elif any(kw in title for kw in ["data", "analytics", "analyst", "scientist", "bi", "insights"]):
        return "Tech"
    elif any(kw in title for kw in ["consultant", "consulting", "operations", "strategy", "project manager", "business", "founder", "ceo", "cfo", "coo", "president"]):
        return "Business"
    elif any(kw in title for kw in ["finance", "accountant", "accounting", "trading", "investment", "bank", "treasury"]):
        return "Finance"
    elif any(kw in title for kw in ["marketing", "brand", "growth", "social media", "communications", "public relations", "advertising"]):
        return "Marketing"
    elif any(kw in title for kw in ["ux", "design", "designer", "creative", "visual", "illustrator"]):
        return "Design"
    elif any(kw in title for kw in ["media", "production", "entertainment", "journalist", "broadcast", "voice over"]):
        return "Media"
    elif any(kw in title for kw in ["teacher", "professor", "student", "academic", "instructional", "coach", "educator"]):
        return "Education"
    elif any(kw in title for kw in ["health", "clinical", "medical", "nurse", "pharma"]):
        return "Healthcare"
    elif any(kw in title for kw in ["law", "legal", "attorney", "compliance", "judicial", "government", "public sector", "policy"]):
        return "Government / Legal"
    elif any(kw in title for kw in ["nonprofit", "volunteer", "community", "outreach", "development", "donor", "events director"]):
        return "Nonprofit"
    else:
        return "Other"
# Clean Job Titles to ensure no NaN values
# Force conversion from non-string values like NaN or numbers to proper strings
# Clean Job Titles to ensure no NaN values
# Force conversion from non-string values like NaN or numbers to proper strings
df["Job Title"] = df["Job Title"].fillna("").astype(str)
df["Auto Category"] = df["Job Title"].apply(assign_category)

Number of rows fetched: 411
First 5 rows:
                                            Job Title           RecordID
0       Senior Director, Corporate Security @Verizon  rec06mTVFx0w9kNdV
1  Global Capital Solutions @ Ares Management Cor...  rec06pz9a2yyinKyL
2              Data Engineer @ Casual Precision, LLC  rec08cCaKhKKmSlAx
3  Sr. HR Administrative Coordinator @Loyola Mary...  rec0lwAgo2xvuetjA
4  Senior Analyst, Sales Relationship Specialist ...  rec0nI89qZwTMByk9


,Job Title,RecordID
0,"Senior Director, Corporate Security @Verizon",rec06mTVFx0w9kNdV
1,Global Capital Solutions @ Ares Management Cor...,rec06pz9a2yyinKyL
2,"Data Engineer @ Casual Precision, LLC",rec08cCaKhKKmSlAx
3,Sr. HR Administrative Coordinator @Loyola Mary...,rec0lwAgo2xvuetjA
4,"Senior Analyst, Sales Relationship Specialist ...",rec0nI89qZwTMByk9
5,Systems Consultant/Techincal Trainer @Cities D...,rec0oBYbJ9ed2YJg2
6,Producer @ C3 Risk & Insurance Services,rec0qW2RclMkhherl
7,Senior Business Systems Analyst @ Native Foods...,rec0uinUEmalfp4Im
8,Sr. Data Analyst @Origence,rec0zeci28P9qv0Mv
9,Software Engineer @Datavant,rec13GdtIpBi4BuJy


In [ ]:
# Collapsing detailed categories into broader ones to improve model performance
collapse_map = {
    "Tech": "Tech",
    "Analytics": "Tech",
    "Business": "Business",
    "Marketing": "Business",
    "Finance": "Business",
    "Sales": "Business",
    "Design": "Creative",
    "Media": "Creative",
    "Education": "Other",
    "Healthcare": "Other",
    "Government / Legal": "Other",
    "Nonprofit": "Other",
    "Other": "Other"
}

# Creating the final category column
df["Collapsed Category"] = df["Auto Category"].map(collapse_map)
print(df["Collapsed Category"].value_counts())

Collapsed Category
Tech        203
Other        97
Business     96
Creative     15
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Features and target
X = df["Job Title"]
y = df["Collapsed Category"]

# Splitting the data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Training the classifier
clf = LogisticRegression(max_iter=1000, class_weight="balanced")
clf.fit(X_train_vec, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    Business       0.61      0.89      0.72        19
    Creative       1.00      0.67      0.80         3
       Other       0.69      0.90      0.78        20
        Tech       0.93      0.61      0.74        41

    accuracy                           0.75        83
   macro avg       0.81      0.77      0.76        83
weighted avg       0.80      0.75      0.75        83



In [ ]:
# 🔁 Reclassify "Other" rows
other_df = df[df["Collapsed Category"] == "Other"].copy()
other_df["Job Title"] = other_df["Job Title"].fillna("").astype(str)

# Transform and predict
other_vec = vectorizer.transform(other_df["Job Title"])
other_df["Reclassified Category"] = clf.predict(other_vec)

# Confidence scores
probs = clf.predict_proba(other_vec)
other_df["Confidence"] = probs.max(axis=1).round(2)

# Show top uncertain predictions
other_df.sort_values(by="Confidence").head(10)


,Job Title,RecordID,Auto Category,Collapsed Category,Reclassified Category,Confidence
366,Clinical Assistant Professor,rectYVH18St7BOuLV,Education,Other,Other,0.30
297,Land Development Intern @ Conner Homes,rechsoYpqWKKZPz7X,Nonprofit,Other,Other,0.30
103,N/A,recELlDIFkZ25Nxu6,Other,Other,Tech,0.31
120,N/A,recGL5ccDEGL9Gl5X,Other,Other,Tech,0.31
254,N/A,recahLH3nT1P40Gto,Other,Other,Tech,0.31
328,Chief Trek Officer @Just Trek,recminr4GyCzSJPZY,Other,Other,Other,0.32
265,Educational Content Creator,reccDY09ZgfTHHyYV,Other,Other,Other,0.32
192,Implementation Manager @LiveRamp,recRfUc6W0fh6fEZI,Other,Other,Other,0.33
158,Internal Education @LMU,recM5ONaHOYuWHa8W,Other,Other,Tech,0.34
147,"Partner @SGI QC, LLC",recK57DghaaHKBE0P,Other,Other,Other,0.34


In [ ]:
df.loc[other_df.index, "Collapsed Category"] = other_df["Reclassified Category"]

In [ ]:
print(df["Collapsed Category"].value_counts())


Collapsed Category
Tech        207
Business     97
Other        92
Creative     15
Name: count, dtype: int64


In [ ]:
# Filter remaining Others
remaining_others = df[df["Collapsed Category"] == "Other"].copy()
remaining_others["Job Title"] = remaining_others["Job Title"].fillna("").astype(str)
remaining_others_vec = vectorizer.transform(remaining_others["Job Title"])

# Predict again
remaining_others["Predicted Category"] = clf.predict(remaining_others_vec)
remaining_probs = clf.predict_proba(remaining_others_vec)
remaining_others["Confidence"] = remaining_probs.max(axis=1).round(2)

# Reclassify if prediction ≠ "Other" and confidence ≥ 0.28
remaining_others["Reclassified Category"] = remaining_others.apply(
    lambda row: row["Predicted Category"] if row["Predicted Category"] != "Other" and row["Confidence"] >= 0.28 else row["Collapsed Category"],
    axis=1
)

# Apply the reclassification
df.loc[remaining_others.index, "Collapsed Category"] = remaining_others["Reclassified Category"]


In [ ]:
df["Collapsed Category"].value_counts()


,count
Collapsed Category,
Tech,207
Business,97
Other,92
Creative,15


In [ ]:
# Second pass on remaining Others with lower threshold
second_pass = df[df["Collapsed Category"] == "Other"].copy()
second_pass["Job Title"] = second_pass["Job Title"].fillna("").astype(str)
second_pass_vec = vectorizer.transform(second_pass["Job Title"])

second_pass["Predicted Category"] = clf.predict(second_pass_vec)
second_probs = clf.predict_proba(second_pass_vec)
second_pass["Confidence"] = second_probs.max(axis=1).round(2)

# Use even lower threshold (0.25) for second pass
second_pass["Reclassified Category"] = second_pass.apply(
    lambda row: row["Predicted Category"] if row["Predicted Category"] != "Other" and row["Confidence"] >= 0.25 else row["Collapsed Category"],
    axis=1
)

# Apply again
df.loc[second_pass.index, "Collapsed Category"] = second_pass["Reclassified Category"]

# See updated counts
print(df["Collapsed Category"].value_counts())


Collapsed Category
Tech        207
Business     97
Other        92
Creative     15
Name: count, dtype: int64


In [ ]:
import json
import time

# Prepare update payload using final categories
records_to_update = []
for i, row in df.iterrows():
    records_to_update.append({
        "id": row["RecordID"],
        "fields": {
            "PredictedCategory": row["Collapsed Category"]  # 👈 final column you're pushing
        }
    })

# Push updates in batches of 10
batch_size = 10
for i in range(0, len(records_to_update), batch_size):
    batch = {"records": records_to_update[i:i + batch_size]}
    response = requests.patch(url, headers=headers, data=json.dumps(batch))
    if response.status_code != 200:
        print(f"❌ Error in batch {i}-{i+batch_size}: {response.json()}")
    time.sleep(0.2)

print("✅ All updates pushed to Airtable!")


✅ All updates pushed to Airtable!
